# Model Serving with Pyfunc Example

This notebook contains a toy example to serve a previously logged model as a python function. In this example, we will use the already trained Elasticnet models using the wine quality dataset from the UCI repository.

The first step is to import all the required libraries. As you can see, there is no need to import model's library (scikit-learn)

In [11]:
import mlflow
from sklearn.model_selection import train_test_split
import pandas as pd
import warnings
import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

warnings.filterwarnings("ignore")

In [ ]:
mlflow.set_tracking_uri("http://localhost:80")

We will download the wine quality dataset to use the data to make predictions.

In [ ]:
# Read the wine-quality csv file from the URL
csv_url = (
    "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
)
try:
    data = pd.read_csv(csv_url, sep=";")
except Exception as e:
    logger.exception(
        "Unable to download training & test CSV, check your internet connection. Error: %s", e
    )

We then extract the train and test data. We will use only test data for the predictions.

In [ ]:
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

print('Train shape:', train_x.shape)
print('Train shape:', test_x.shape)

Load the model previously stored as a python function. User the artifact-uri + '/model' to load de model

In [ ]:
loaded_pyfunc = mlflow.pyfunc.load_model('s3://mlflow-bucket/mlflow/5/f33c369110a14a6d813eef12da1f1273/artifacts/model')

Make predictions using the loaded model and the test data (features)

In [ ]:
loaded_pyfunc.predict(test_x)